In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
from concurrent.futures import ThreadPoolExecutor
import os
import pandas as pd
from tqdm import tqdm

workers = os.cpu_count()

headers = {
    "authority": "www.farmagora.com.br",
    "accept": "*/*",
    "accept-language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "referer": "https://www.farmagora.com.br/",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
}

### Extraindo o link da API dos produtos

In [ ]:
blocos = ['medicamentos', 'saude-e-bem-estar', 'dermocosmeticos', 'mamae-e-bebe', 'higiene', 'primeiros-socorros', 'dieta-e-suplemento', 'aparelhos', 'perfumes', 'conveniencia']
categorias = []
for bloco in blocos:
    url = f'https://www.farmagora.com.br/{bloco}?lid=e9f374f7-7fb9-4504-b671-e95f400118cf'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    for h4_tag in soup.find_all('h4', class_=f'{bloco}'):
        category_name = h4_tag.text.strip()
        category_url = h4_tag.a['href']

        categorias.append({"Categoria": category_name, "URL": category_url})

In [ ]:
blocos_formatados = [
    'Medicamentos',
    'Saúde e bem estar',
    'Dermocosméticos',
    'Mamãe e bebê',
    'Higiene',
    'Primeiros socorros',
    'Dieta e suplemento',
    'Aparelhos',
    'Perfumes',
    'Conveniência']

categorias_filtradas = [categoria for categoria in categorias if categoria['Categoria'] not in blocos_formatados]

# Criar DataFrame
df_categorias = pd.DataFrame(categorias_filtradas)

df_categorias

In [ ]:
ids = []

for url in df_categorias['URL']:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        script_content = soup.find('script', string=re.compile('vtxctx'))
        if script_content:
            match = re.search(r'vtxctx = ({.*?});', script_content.string)
            if match:
                json_content = match.group(1)

                # Analisar o JSON para um dicionário Python
                data = json.loads(json_content)

                department_id = data.get('departmentyId', None)
                category_id = data.get('categoryId', None)

                ids.append({"URL": url, "Department ID": department_id, "Category ID": category_id})
            else:
                print(f"Padrão não encontrado no script da URL: {url}")
        else:
            print(f"Script não encontrado na URL: {url}")
    else:
        print(f"Erro na requisição. Código de status: {response.status_code}")

df_ids = pd.DataFrame(ids)

In [ ]:
links_produtos = []

for id in lista_ids:
    department_id = id['Department ID']
    category_id = id['Category ID']

    page = 1
    while page <= 50:  # Defina um limite para evitar loops infinitos
        url = f'https://www.farmagora.com.br/buscapagina?fq=C:/{department_id}/{category_id}/&PS=24&sl=ba7aedf8-c57e-44a0-ba5c-83285d3577a3&cc=24&sm=0&PageNumber={page}&O=OrderByTopSaleDESC'
        response = requests.get(url, headers=headers)

        # Verifica se a requisição foi bem-sucedida
        if response.status_code == 200 or response.status_code == 206:
            # Parseia o conteúdo HTML
            soup = BeautifulSoup(response.text, 'html.parser')

            # Encontrar os data-product-ids
            product_links = soup.select('.product-item__title a')
            if not product_links:
                break  # Sai do loop se não houver mais produtos

            for link in product_links:
                product_href = link['href'].split('/')[-2]
                links_produtos.append(product_href)
            # Aumenta o número da página
            print(category_id, page)
            page += 1
        else:
            print(f"Erro na requisição. Código de status: {response.status_code}")
            break  # Sai do loop em caso de erro

lista_href = list(set(links_produtos))
df_links = pd.DataFrame(lista_href)
df_links.to_csv('df_links.csv')

### Extraindo as informações

In [8]:
df_links = pd.read_csv('df_links.csv', index_col=0)  # Use index_col=0 para usar a primeira coluna como índice
df_links.columns = ['href']


In [9]:
def extrair_infos(href):
    try:
        api_url = f'https://www.farmagora.com.br/api/catalog_system/pub/products/search/{href}/p'
        response = requests.get(api_url, headers=headers)
        if response.status_code == 200:
            product_data_list = response.json()
            produtos = []

            for product_data in product_data_list:
                nome = product_data.get('productName', '')
                ean = product_data.get('items', [{}])[0].get('ean', '')
                marca = product_data.get('brand', '')
                
                descricao_raw = product_data.get('description', '')
                soup = BeautifulSoup(descricao_raw, 'html.parser')
                descricao_ = soup.get_text(separator=' ', strip=True)
                descricao = limpar_texto(descricao_)

                preco_s = product_data.get('items', [{}])[0].get('sellers', [{}])[0].get('commertialOffer', {}).get('ListPrice', 0.0)
                preco_c = product_data.get('items', [{}])[0].get('sellers', [{}])[0].get('commertialOffer', {}).get('Price', 0.0)
                porcentagem = round(100 * (1 - (preco_c / preco_s))) if preco_s > 0 else 0.0

                # Construir o dicionário com as informações e adicionar à lista
                info = {
                    'Produto': nome,
                    'EAN': ean,
                    'Marca': marca,
                    'Descrição': descricao,
                    'Preço sem desconto': preco_s,
                    'Preço com desconto': preco_c,
                    '% de desconto': f'{porcentagem}%',
                    'Farmácia': 'Preço Popular',
                    'Região': 'Sudeste',
                    'Cidade': 'São Paulo'
                }
                produtos.append(info)
                pbar.update(1)

            return produtos

        print(f"Nenhum produto encontrado para {href}")
        return None
    except Exception as e:
        print(f"Erro ao processar {href}: {str(e)}")
        return None
    
def limpar_texto(texto):
    # Substituir caracteres especiais de pontuação por espaços
    texto = re.sub(r'[^\w\s.,;:!?()\[\]{}-]', ' ', texto)
    # Substituir espaços múltiplos por um único espaço
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

# Criar uma lista para armazenar os resultados
infos = []

# Inicializar o tqdm
with tqdm(total=len(df_links)) as pbar:
    with ThreadPoolExecutor(max_workers=workers) as executor:
        # Usar o método map corretamente
        results = executor.map(extrair_infos, df_links['href'])
        
        # Filtrar resultados para remover valores nulos
        infos = [info for info in results if info is not None]

# Converter a lista de dicionários em um DataFrame do pandas
df_infos = pd.DataFrame([item[0] for item in infos])
df_infos.to_csv('df_infos.csv', index=False)

  4%|▍         | 525/12779 [00:59<55:06,  3.71it/s]  

Nenhum produto encontrado para barra-bold-thin-40gr-caramelo-<->-<->-amendoim


 17%|█▋        | 2112/12779 [06:01<29:23,  6.05it/s]  

Nenhum produto encontrado para creme-dental-sensodyne-com-2x100gr-repair-<->-<->-protect


 36%|███▌      | 4599/12779 [13:51<25:40,  5.31it/s]  

...

 76%|███████▋  | 9758/12779 [29:58<10:55,  4.61it/s]

Nenhum produto encontrado para miosan-caf-5-30mg-com-15-comprimidos


 98%|█████████▊| 12541/12779 [38:25<00:51,  4.64it/s]

Nenhum produto encontrado para des-nivea-masc-200ml-aero-b<->-<->w-invisible


100%|█████████▉| 12771/12779 [39:05<00:01,  5.45it/s]
